In [3]:
%run "SparkApp.ipynb"

24/12/21 14:59:04 WARN Utils: Your hostname, Ala resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/21 14:59:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ala/.ivy2/cache
The jars for the packages stored in: /home/ala/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-578bef22-520f-4d7e-80b2-83f602c72739;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution

In [4]:
spark

In [18]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType

schema = StructType([
    StructField("id", StringType(), True),
    StructField("vehicle", StructType([
        StructField("trip", StructType([
            StructField("tripId", StringType(), True),
            StructField("startTime", StringType(), True),
            StructField("startDate", StringType(), True),
            StructField("scheduleRelationship", StringType(), True),
            StructField("routeId", StringType(), True),
            StructField("directionId", LongType(), True),
        ]), True),
        StructField("position", StructType([
            StructField("latitude", DoubleType(), True),
            StructField("longitude", DoubleType(), True),
            StructField("bearing", DoubleType(), True),
            StructField("speed", DoubleType(), True),
        ]), True),
        StructField("currentStopSequence", LongType(), True),
        StructField("currentStatus", StringType(), True),
        StructField("timestamp", StringType(), True),
        StructField("congestionLevel", StringType(), True),
        StructField("stopId", StringType(), True),
        StructField("vehicle", StructType([
            StructField("id", StringType(), True),
            StructField("label", StringType(), True),
            StructField("licensePlate", StringType(), True),
        ]), True),
    ]), True),
])


In [19]:
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "vehicleposition") \
    .option("startingOffsets","earliest") \
    .load()

In [20]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [21]:
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [22]:
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data"))

In [23]:
streaming_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- vehicle: struct (nullable = true)
 |    |    |-- trip: struct (nullable = true)
 |    |    |    |-- tripId: string (nullable = true)
 |    |    |    |-- startTime: string (nullable = true)
 |    |    |    |-- startDate: string (nullable = true)
 |    |    |    |-- scheduleRelationship: string (nullable = true)
 |    |    |    |-- routeId: string (nullable = true)
 |    |    |    |-- directionId: long (nullable = true)
 |    |    |-- position: struct (nullable = true)
 |    |    |    |-- latitude: double (nullable = true)
 |    |    |    |-- longitude: double (nullable = true)
 |    |    |    |-- bearing: double (nullable = true)
 |    |    |    |-- speed: double (nullable = true)
 |    |    |-- currentStopSequence: long (nullable = true)
 |    |    |-- currentStatus: string (nullable = true)
 |    |    |-- timestamp: string (nullable = true)
 |    |    |-- congestionLevel: string (nullable = true)

In [24]:
flattened_df = streaming_df.select(
    col("data.id").alias("id"),
    col("data.vehicle.trip.tripId").alias("tripId"),
    col("data.vehicle.trip.startTime").alias("startTime"),
    col("data.vehicle.trip.startDate").alias("startDate"),
    col("data.vehicle.trip.scheduleRelationship").alias("scheduleRelationship"),
    col("data.vehicle.trip.routeId").alias("routeId"),
    col("data.vehicle.trip.directionId").alias("directionId"),
    col("data.vehicle.position.latitude").alias("latitude"),
    col("data.vehicle.position.longitude").alias("longitude"),
    col("data.vehicle.position.bearing").alias("bearing"),
    col("data.vehicle.position.speed").alias("speed"),
    col("data.vehicle.currentStopSequence").alias("currentStopSequence"),
    col("data.vehicle.currentStatus").alias("currentStatus"),
    col("data.vehicle.timestamp").alias("timestamp"),
    col("data.vehicle.congestionLevel").alias("congestionLevel"),
    col("data.vehicle.stopId").alias("stopId"),
    col("data.vehicle.vehicle.id").alias("vehicleId"),
    col("data.vehicle.vehicle.label").alias("vehicleLabel"),
    col("data.vehicle.vehicle.licensePlate").alias("licensePlate"),
)

In [25]:
flattened_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tripId: string (nullable = true)
 |-- startTime: string (nullable = true)
 |-- startDate: string (nullable = true)
 |-- scheduleRelationship: string (nullable = true)
 |-- routeId: string (nullable = true)
 |-- directionId: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- bearing: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- currentStopSequence: long (nullable = true)
 |-- currentStatus: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- congestionLevel: string (nullable = true)
 |-- stopId: string (nullable = true)
 |-- vehicleId: string (nullable = true)
 |-- vehicleLabel: string (nullable = true)
 |-- licensePlate: string (nullable = true)



In [26]:
analysis_df = flattened_df.groupBy("routeId").count()

In [27]:
query = analysis_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

24/12/21 15:07:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-be2b56fc-bec6-4715-9a3e-c78c3ffa1d70. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/21 15:07:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/21 15:07:42 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/12/21 15:07:42 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/12/21 15:07:42 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/12/21 15:07:42 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known con

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----+
|routeId|count|
+-------+-----+
+-------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----+
| routeId|count|
+--------+-----+
|40A47374|    1|
|  447374|    1|
|  196921|    1|
|   86921|    1|
|   76921|    1|
|  266990|    2|
| 2947374|    2|
|40C47374|    2|
|  356920|    8|
|   96921|    2|
|  610299|    1|
|  146921|    1|
|  386921|    1|
|  156920|    5|
+--------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-----+
| routeId|count|
+--------+-----+
|  116990|    4|
| 8210299|    1|
|  276990|    1|
|  156921|    3|
| 8110299|    2|
| 8547374|    1|
| 7047374|    3|
|40A47374|    2|
| 90X6852|    1|
|  447374|   11|
|  196921|    7|
| 5547374|    2|
|   86921|   14|
| 5047374|    4|
|  606957|    1|
|   76921|    9|
|52B47374|    1|
|  716990|    1|
|  136990|    4|
| 7447374|    1|
+--------+-----+
only showing top 20 rows



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/ala/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ala/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 